In [1]:
# setup
import pandas as pd
# import statsmodels.formula.api as smf
# import statsmodels.api as sm
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
import tprstats
solar = pd.read_excel("../data/Solar_Data.xlsx").dropna()

In [2]:
# use C(DrivesHybrid) to test Logit works with categorical variables
solar_logit=tprstats.model("logit",formula="Sale~AnnualSave+AnnualCost+C(DrivesHybrid)+NearbySolar+HomeValue",data=solar)
solar_logit.summary()

Optimization terminated successfully.
         Current function value: 0.102926
         Iterations 9


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                   Sale   No. Observations:                 1500
Model:                          Logit   Df Residuals:                     1494
Method:                           MLE   Df Model:                            5
Date:                Tue, 26 Nov 2024   Pseudo R-squ.:                  0.2847
Time:                        20:14:56   Log-Likelihood:                -154.39
converged:                       True   LL-Null:                       -215.84
Covariance Type:            nonrobust   LLR p-value:                 7.619e-25
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept              -12.7445      4.837     -2.635      0.008     -22.224      -3.265
C(DrivesHybrid)[T.1]     0.9644      0.413      2.333      0.020       0.154       1.775
AnnualSave               3.9268      0.649      6.048      0.000       2.654       5.199
AnnualCost              -4.0989      1.475     -2.778      0.005      -6.990      -1.207
NearbySolar              1.9421      0.398      4.884      0.000       1.163       2.722
HomeValue                0.0178      0.008      2.192      0.028       0.002       0.034
========================================================================================
"""

In [3]:
solar_logit.predict()

array([0.01617376, 0.00438152, 0.02321191, ..., 0.1497396 , 0.0078212 ,
       0.02681689])

In [4]:
solar_classification_table = solar_logit.classification_table()
solar_classification_table

[1176.0, 275.0, 6.0, 43.0]


,Summary,Actual,Predicted,Frequency
0,Correct,0,0,1176.0
1,Incorrect,0,1,275.0
2,Incorrect,1,0,6.0
3,Correct,1,1,43.0


In [5]:

X_out_of_sample = pd.DataFrame({
    "AnnualSave": [2.5, 1.9, 2.3],
    "AnnualCost": [1.8, 1.5, 1.7],
    "DrivesHybrid": [1,0,0],
    "NearbySolar": [0, 1, 0],
    "HomeValue": [550, 675, 495]
})
print("We create three homes: ")
print(X_out_of_sample)
print("Predicted Probabilities of purchase for the three homes are: ")
print(solar_logit.predict(X_out_of_sample))

We create three homes: 
   AnnualSave  AnnualCost  DrivesHybrid  NearbySolar  HomeValue
0         2.5         1.8             1            0        550
1         1.9         1.5             0            1        675
2         2.3         1.7             0            0        495
Predicted Probabilities of purchase for the three homes are: 
0    0.613822
1    0.927109
2    0.135068
dtype: float64


In [6]:
# marginal effects
solar_logit.marginal_effects()

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:                   Sale
Method:                          dydx
At:                           overall
========================================================================================
                          dy/dx    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
C(DrivesHybrid)[T.1]     0.0249      0.011      2.312      0.021       0.004       0.046
AnnualSave               0.1013      0.018      5.535      0.000       0.065       0.137
AnnualCost              -0.1057      0.039     -2.721      0.007      -0.182      -0.030
NearbySolar              0.0501      0.011      4.611      0.000       0.029       0.071
HomeValue                0.0005      0.000      2.161      0.031    4.27e-05       0.001
========================================================================================
"""

In [ ]:
solar_new = pd.read_excel("../data/Solar_New_Prospects.xlsx")
solar_new['PredNew'] = solar_logit.predict(solar_new)
solar_new["ProspectRank"] = solar_new["PredNew"].rank()
solar_new

,Customer,AnnualSave,ElecBill,AnnualCost,DrivesHybrid,NearbySolar,HomeValue,PredNew,ProspectRank
608,609,0.607246,0.850120,1.831445,0,0,536.098088,0.000246,1.0
54,55,0.664754,0.735625,1.641288,0,0,508.298030,0.000409,2.0
935,936,0.863014,1.234059,1.756444,0,0,496.592665,0.000451,3.0
385,386,1.111673,1.384965,2.022553,0,0,506.252013,0.000478,4.0
999,1000,0.708387,0.735625,1.641288,0,0,508.298030,0.000486,5.0
...,...,...,...,...,...,...,...,...,...
817,818,3.181619,2.468633,1.811254,0,0,513.716687,0.814978,996.0
871,872,2.900204,2.835939,1.718183,0,0,555.587322,0.818398,997.0
850,851,2.792388,1.887400,1.942580,0,1,525.557723,0.827666,998.0
938,939,3.244934,2.513898,2.034931,0,0,556.284208,0.828255,999.0
